# Data Quality Plan
Analysing the Dublin Bus Data provided to optimise model performance

## Summary
Each file has about 750,000 rows. Three quarters of a million.

## Ideas to handle data.
1. __Incremental Handing__
__Route Division__
Logic: Partition data into subsets of data which are internally comparable.
         Does it make sense that a single model should describe all routes?
         Since routes are different they should be modeled individually.

A machine can maintain 12 DF max in a single object. (atleast mine can)
If instead we extract each route from the data to build a dataframe of a single 
route. Then model it, take the best model. Now we have 100th of the data to handle in each increment.

When we have derived the model, we store it and its route, but can discard the data then. 

If we create a function that returns the model. We can either use re-assignment or scoping discard the dataframe in python.

For each route we store a model which is the sum of the models for the sections in which it is composed.

__Section Division__
Logic: The sum of the parts gives the value of the whole.

Many smaller data sets. This models the travel time for different sections in which the busses pass through. This reduces redundancy in the analysis of bus routes (which overlap)

Though since there are more sections than bus routes, there will be many more models to store, problem then becomes iterative. given t_0 the time we , we can calculate t_1 (the time we arrive in section1 after traversing seciont_0) then we apply the model for section 1 with t_1 as an input. (recirsive problem, easily described iteritavly)

the total time is the sum of all the predicted times for each section.

### Task
Create a function that tests different models and chooses the best.
Run this for each section storing each model. Then given a departure point and a destination we can derive the appropriate route and the sum of its sections.

How to derive the route. How often do time stamps occur. Accuracy of estimation of length of a section depends on the frequancy in which a time stamp is put out. Assume a section takes 100.5 seconds to cross. if a stamp is put out every second and there are 100 time stamps in a journey we know that the time taken is between 100 and 101 (small percentage error). The best estimate that can be made is that that time taken is in between 100 and 102.

If a time stamp occurs only every 50 seconds, then in a 100 second interval there could be 2 or 3 time stamps. i.e. we will predict it takes between 100 and 150.

Max-Min <= x <= (n+1/n)*(Max-Min), where n is the number of timestamps for a specific journey id occuring within a section)



##
2. __


We can use sparse data, take only every third entry for a given bus on a route. We then can fit the whole frame?


Best approach may be to develop models route by route, then save sparse data for increasing performance (if nexesary?)

sql query to return every 30th row, (or nth row) to reduce data if needed.
SELECT t.id, t.key
FROM	
(
    SELECT id, key, ROW_NUMBER() OVER (ORDER BY key) AS rownum
    FROM datatable
) AS t
WHERE t.rownum % 30 = 0    -- or % 40 etc
ORDER BY t.key




In [1]:
#import sections
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from patsy import dmatrices
import matplotlib.patches as mpatches
import statsmodels.formula.api as sm
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from statsmodels.formula.api import logit
from sklearn.cross_validation import train_test_split

import os
import glob


C:\Users\Andy\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### This was the code used to merge route 4

# Right now these functions are only built on one level, but they will be modularised and optimised by the time we build the MASTER function

In [2]:
# #method 1, ends at 12 dataframes.

# path=os.getcwd() + "\\data"

# def merge_frames(path):
#     column_names = ["Timestamp",
#                     "LineID", 
#                     "Direction",
#                     "Journey_Pattern_ID", 
#                     "Timeframe", 
#                     "Vehicle_Journey_ID", 
#                     "Operator", 
#                     "Congestion", 
#                     "Lon",
#                     "Lat", 
#                     "Delay", 
#                     "Block_ID",
#                     "Vehicle_ID",
#                     "Stop_ID",
#                     "At_Stop"]
    
#     #should help loop run a little faster.
#     read = pd.read_csv
#     concat = pd.concat
    
#     #stores contents of a folder as a list.
#     contents = os.listdir(path) 
#     content_length = len(contents)
    
    
#     #reads csv from data folder in cwd
#     accumulator = read(path+"\\"+contents[0], index_col=None, header=0, encoding="utf-8")
#     accumulator.columns = column_names
    
#     for i in range(content_length):

#         next_df = pd.read_csv(path+"\\"+contents[i], index_col=None, header=0)
#         next_df.columns = column_names

#         accumulator = pd.concat([accumulator[(accumulator["LineID"]==4)], next_df[(next_df["LineID"]==4)]], axis=0)
#         print(accumulator.shape, "acc")
        
#     return accumulator

# df_1 = merge_frames(path)

# df_1.shape


In [ ]:
#This was the code used to get route data for all files by lineID

In [3]:

# path=os.getcwd() + "\\data"

# def extract_merge_frames(path):
#     column_names = ["Timestamp",
#                     "LineID", 
#                     "Direction",
#                     "Journey_Pattern_ID", 
#                     "Timeframe", 
#                     "Vehicle_Journey_ID", 
#                     "Operator", 
#                     "Congestion", 
#                     "Lon",
#                     "Lat", 
#                     "Delay", 
#                     "Block_ID",
#                     "Vehicle_ID",
#                     "Stop_ID",
#                     "At_Stop"]
    
    
#     #stores contents of a folder as a list.
#     contents = os.listdir(path) 
#     content_length = len(contents)
    
    
#     #reads csv from data folder in cwd
#     read = pd.read_csv
#     accumulator = read(path+"\\"+contents[0], index_col=None, header=0)
#     accumulator.columns = column_names
#     set_of_routes = set(accumulator.LineID.unique())
#     comparison_set = set_of_routes
    
#     #should help loop run a little faster.
#     concat = pd.concat
#     unite = set_of_routes.union
    
        
#     #read through all files once and determine maximum set of routes
    
#     for i in range(1,content_length):
#         next_df = pd.read_csv(path+"\\"+contents[i], index_col=None, header=0)
#         next_df.columns = column_names
#         next_df.columns = column_names
#         next_set_of_routes = set(next_df.LineID.unique())
#         set_of_routes = unite(next_set_of_routes)
#         print("routes extracted from file:", i)
    
#     print("total set of routes\n", set(set_of_routes)==set(comparison_set))
#     print(comparison_set - set_of_routes)
#     print(set_of_routes - comparison_set)
    
#     for route in set_of_routes: 
#         try:
#             print(route)
#             int(route) #if i get past this op, i can continue

#             for i in range(1, content_length):
#                 print("extracting route:",route,"\tfrom file ",i)
#                 next_df = pd.read_csv(path+"\\"+contents[i], index_col=None, header=0)
#                 next_df.columns = column_names
#                 accumulator = pd.concat([accumulator[(accumulator["LineID"]==route)], next_df[(next_df["LineID"]==route)]], axis=0)
        
#             accumulator.to_csv( "route-" + str(int(route)) + "-raw1.csv", encoding = "utf-8" )
        
#             print ("route:", route , "complete\n")
        
#         except:
#             print("You just got NaN-ed!")
        
        
            
#     return accumulator #i return is so i can assess the dataframe after

# df_1 = extract_merge_frames(path)

# This is the code used on the November data, which has different quality

In [5]:
# # extra routes 32, 142, 111,114, 51, 116 , 118

# path=os.getcwd() + "\\by_day-Nov2012"

# def extract_merge_frames(path):
#     column_names = ["Timestamp",
#                     "LineID", 
#                     "Direction",
#                     "Journey_Pattern_ID", 
#                     "Timeframe", 
#                     "Vehicle_Journey_ID", 
#                     "Operator", 
#                     "Congestion", 
#                     "Lon",
#                     "Lat", 
#                     "Delay", 
#                     "Block_ID",
#                     "Vehicle_ID",
#                     "Stop_ID",
#                     "At_Stop"]
    
    
#     #stores contents of a folder as a list.
#     contents = os.listdir(path) 
#     content_length = len(contents)
    
    
#     #reads csv from data folder in cwd
#     read = pd.read_csv
    
#     accumulator = read(path+"\\"+contents[0], index_col=None, header=0)
#     accumulator.columns = column_names
#     accumulator.LineID.astype("str")
    
#     #init for first loop
#     set_of_routes = set(accumulator.LineID.unique())
#     comparison_set = set_of_routes
    
#     #should help loop run a little faster.
#     concat = pd.concat
#     unite = set_of_routes.union
    
        
#     #read through all files once and determine maximum set of routes for partitioning

    
#     for i in range(1,content_length):
#         next_df = pd.read_csv(path+"\\"+contents[i], index_col=None, header=0)
#         next_df.columns = column_names
#         next_df.LineID.astype("str")
#         next_set_of_routes = set(next_df.LineID.unique())
#         set_of_routes = unite(next_set_of_routes)
#         print("routes extracted from file:", i)
    
#     print(comparison_set - set_of_routes)
#     print(set_of_routes - comparison_set)
#     set_of_routes = sorted(set_of_routes, key = str)
#     print(len(set_of_routes), set_of_routes)
    
#     intset = {str(x) for x in set_of_routes if isinstance(x, int)}
#     strset = {x for x in set_of_routes if isinstance(x, str)}
#     set_of_routes = sorted(intset.union(strset), key=str)
#     print("new", set_of_routes)
    
#     for route in set_of_routes: 
#         accumulator = read(path+"\\"+contents[0], index_col=None, header=0)
#         accumulator.columns = column_names
#         accumulator.LineID.astype("str")
#         try:
#             print(route)
#             print(str(route))#if

#             for i in range(1, content_length):
#                 print("extracting route:",route,"\tfrom file ",i)
#                 next_df = pd.read_csv(path+"\\"+contents[i], index_col=None, header=0)
#                 next_df.columns = column_names
#                 next_df.LineID.astype("str")
                
#                 accumulator = concat([accumulator[(accumulator["LineID"]==route)], next_df[(next_df["LineID"]==route)]], axis=0)
        
#             #used random to make sure some files dont overwrite eachoter...
#             accumulator.to_csv( "route-" + route + "-raw2.csv", encoding = "utf-8" )
        
#             print ("route:", route , "complete\n")
        
#         except:
#             print("You just got NaN-ed!")
        
        
            
#     return accumulator #i return is so i can assess the dataframe after


# df_1 = extract_merge_frames(path)



# This is where i plan to build the master function

### Master function should be build using stepwise refinement, it needs to strong and tested very well, it will be expensive to run, so lets make sure it does it correctly.


This function has to read in raw- original csv data, parition it into the appropriate route info files
re-derive some column entries, drop duplicates, re-build or drop nulls and must do, save data in utf8 format etc.




this is the function that will partition, clean and save new csv files for analysis.
its based off the function above (obviously) but contains small snippets of psuedo code and instructions.
Replace line ID

In [ ]:
# # extra routes 32, 142, 111,114, 51, 116 , 118

# path=os.getcwd() + "\\by_day-Nov2012"

# def extract_merge_frames(path):
#     column_names = ["Timestamp",
#                     "LineID", 
#                     "Direction",
#                     "Journey_Pattern_ID", 
#                     "Timeframe", 
#                     "Vehicle_Journey_ID", 
#                     "Operator", 
#                     "Congestion", 
#                     "Lon",
#                     "Lat", 
#                     "Delay", 
#                     "Block_ID",
#                     "Vehicle_ID",
#                     "Stop_ID",
#                     "At_Stop"]
    
    
#     #stores contents of a folder as a list.
#     contents = os.listdir(path) 
#     content_length = len(contents)
    
    
#     #reads csv from data folder in cwd
#     read = pd.read_csv
#     accumulator = read(path+"\\"+contents[0], index_col=None, header=0)
#     accumulator.columns = column_names
#     accumulator.Journey_Pattern_ID.astype("str")
    
    
#     #init for first loop
#     set_of_routes = set(accumulator.Journey_Pattern_ID.unique())
#     comparison_set = set_of_routes
    
#     #should help loop run a little faster.
#     concat = pd.concat
#     unite = set_of_routes.union
    
        
#     #read through all files once and determine maximum set of routes
    
#     for i in range(1,content_length):
#         next_df = pd.read_csv(path+"\\"+contents[i], index_col=None, header=0)
#         next_df.columns = column_names
#         next_df.Journey_Pattern_ID.astype("str")
#         next_set_of_routes = set(next_df.Journey_Pattern_ID.unique())
#         set_of_routes =  unite(next_set_of_routes)
#         print("routes extracted from file:", i)
    
#     #Decision do we break routes on their directionality
    
#     """ read journey patter and slice first 4 digits (5 bits if you want to include direction)
#     while leading zeros: remove zeros
#     put remaining bits back into set"""
    
    
#     #Run set comparison for debugging
#     print(comparison_set - set_of_routes)
#     print(set_of_routes - comparison_set)
#     set_of_routes = sorted(set_of_routes, key = str)
#     print(len(set_of_routes), set_of_routes)
    
    
#     intset = {str(x) for x in set_of_routes if isinstance(x, int)}
#     strset = {x for x in set_of_routes if isinstance(x, str)}
#     set_of_routes = sorted(intset.union(strset), key=str)
#     print("new", set_of_routes)
    
    
#     for route in set_of_routes: 
#         accumulator = read(path+"\\"+contents[0], index_col=None, header=0)
#         accumulator.columns = column_names
#         accumulator.LineID.astype("str")
#         try:
#             print(route)
#             print(str(route))#if

#             for i in range(1, content_length):
#                 print("extracting route:",route,"\tfrom file ",i)
#                 next_df = pd.read_csv(path+"\\"+contents[i], index_col=None, header=0)
#                 next_df.columns = column_names
#                 next_df.LineID.astype("str")
                
#                 accumulator = concat([accumulator[(accumulator["LineID"]==route)], next_df[(next_df["LineID"]==route)]], axis=0)
        
#             #used random to make sure some files dont overwrite eachoter...
#             accumulator.to_csv( "route-" + route + "-raw2.csv", encoding = "utf-8" )
        
#             print ("route:", route , "complete\n")
        
#         except:
#             print("You just got NaN-ed!")
        
        
            
#     return accumulator #i return is so i can assess the dataframe after


# df_1 = extract_merge_frames(path)




In [6]:
# df_1.Timestamp.astype("datetime64[ms]")
df_29A = pd.read_csv("route-29-raw2.csv", index_col=0)
df_29A.Journey_Pattern_ID.astype("str")
df_18 = pd.read_csv("route-18-raw2.csv", index_col=0)
# del df_29['Unnamed:0']
# del df_29A['Unnamed:0']
df_29A.head()

C:\Users\Andy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Timestamp,LineID,Direction,Journey_Pattern_ID,Timeframe,Vehicle_Journey_ID,Operator,Congestion,Lon,Lat,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop
122,1352160012000000,29,0,029A1001,2012-11-05,2639,CF,0,-6.190650,53.359283,-259,29007,33238,613,0
274,1352160033000000,29,0,029A1001,2012-11-05,2639,CF,0,-6.190617,53.359299,-259,29007,33238,613,0
459,1352160053000000,29,0,029A1001,2012-11-05,2639,CF,0,-6.190550,53.359482,-259,29007,33238,613,0
601,1352160071000000,29,0,029A1001,2012-11-05,2639,CF,0,-6.190550,53.359482,-259,29007,33238,613,0
743,1352160092000000,29,0,029A1001,2012-11-05,2639,CF,0,-6.190283,53.359299,-259,29007,33238,613,0


## df_18.head()

In [7]:
print(df_29A.Journey_Pattern_ID.unique())
print(df_18.Journey_Pattern_ID.unique())

['029A1001' 'null' '029A0001' '00180001' nan]
['00181001' '00180001' 'null' '00181002' nan '01451001']


In [8]:
# open dataframe of 31 data. view unique bus stops where journey_parrern_id equal 31.
#check that every journey pattern id in that file has that sequence of bus stops

#check that every 32000xxx has a different sequence.

stops_29 = set(df_29A[df_29A["Journey_Pattern_ID"]=="029A1001"].Stop_ID.unique())
print(stops_29)

{'738', '741', '4804', 5154, 5157, '742', '658', '745', '617', '5157', '609', 600, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, '950', 621, 620, '616', '605', '655', '604', '659', '608', '618', 654, 655, 656, 657, 658, '611', 659, 675, '291', '612', 4800, 4804, '7551', '915', '739', '744', 738, 739, '740', 741, 742, 743, 744, 745, '613', 740, '615', '621', 291, 296, '606', '620', '657', '5154', '916', '619', '600', '656', '525', '614', 7551, '610', '918', 915, 916, 917, 918, 919, '917', '4800', 950, '296', '919', '607', '743', '675', '654'}


In [9]:
stops_18 = set(df_18[df_18["Journey_Pattern_ID"]=="00181001"].Stop_ID.unique())

In [10]:
# I use this code to prove using signitures that journey pattern ids contain more accurate route info that Line_id


print(stops_18 == stops_29)
print(stops_18 - stops_29)
print(stops_29 - stops_18)
print(stops_29.intersection(stops_18))
print(len(stops_29.union(stops_18))== len(stops_29) + len(stops_18))

stops_29A1 = set(df_29A[df_29A["Journey_Pattern_ID"]=="029A1001"].Stop_ID.unique())

intset = {x for x in stops_29A1 if isinstance(x, int)}
strset = {int(x) for x in stops_29A1 if isinstance(x, str)}
stops_29A1 = set(sorted(intset.union(strset), key=str))

stops_29A2 = set(df_29A[df_29A["Journey_Pattern_ID"]=="029A0001"].Stop_ID.unique())
intset = {x for x in stops_29A2 if isinstance(x, int)}
strset = {int(x) for x in stops_29A2 if isinstance(x, str)}
stops_29A2 = set(sorted(intset.union(strset), key=str))


# print((stops_29A1))
# print("")
# print((stops_29A2))
# print("")

print("")
print("")
print(stops_29A1 == stops_29A2)
print(stops_29A1 - stops_29A2)
print(stops_29A2 - stops_29A1)
print()
print(stops_29A2.intersection(stops_29A1))
print(len(stops_29A2.union(stops_29A1)) == len(stops_29A2) + len(stops_29A1))


False
{'7526', '2781', '2655', '2482', '2331', '753', '850', '486', '2479', '2487', '5026', '7529', '2206', '2208', '2727', '2726', '2793', '1077', '2211', '7530', '2212', '487', '2808', '2105', '7525', '2656', '2672', '2480', '2485', '2101', '2448', '385', '2653', '2654', '2782', '2330', '2798', '851', '2489', '2780', '2481', '2484', '2807', '2806', '4888', '754', '2791', '4357', '7527', '2488', '7165', '2207', '2799', '2783', '2102', '2204', '2794', '2210', '849', '4359', '755', '2486', '2668', '2673', '2652', '2328', '2483', '2696', '2688', '2329', '2689', '2450', '7528'}
{'738', '741', '4804', 5154, 5157, '742', '658', '745', '617', '5157', '609', 600, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, '950', 621, 620, '616', '605', '655', '604', '659', '608', '618', 654, 655, 656, 657, 658, '611', 659, 675, '291', '612', 4800, 4804, '7551', '915', '739', '744', 738, 739, 740, 741, '740', '613', 742, 743, 744, 745, '615', '621', 291, 296, '606', '620', 

In [11]:
# df_1.Timestamp.astype("datetime64[ms]")
df_31 = pd.read_csv("route-32-raw2.csv", index_col=0)
# del df_29['Unnamed:0']
# del df_29A['Unnamed:0']
df_31.head()
#if its the same vehicle between in same journey_id then extrapolate the journey pattern id.

,Timestamp,LineID,Direction,Journey_Pattern_ID,Timeframe,Vehicle_Journey_ID,Operator,Congestion,Lon,Lat,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop
13,1352160002000000,32,0,null,2012-11-05,2757,CF,0,-6.246833,53.354599,775,32007,38083,3586,0
78,1352160008000000,32,0,00320001,2012-11-05,2721,CF,0,-6.124481,53.433102,65,32003,38079,3604,0
162,1352160023000000,32,0,null,2012-11-05,2757,CF,0,-6.245217,53.355850,775,32007,38083,3586,0
220,1352160027000000,32,0,00320001,2012-11-05,2721,CF,0,-6.123217,53.435215,65,32003,38079,3604,0
351,1352160041000000,32,0,null,2012-11-05,2757,CF,0,-6.242033,53.358032,775,32007,38083,3586,0


In [12]:
print(df_31.Journey_Pattern_ID.unique())

['null' '00320001' '00321001' '032B1001' '032A1001' '032B0001' '032A0001'
 '032X0001' '032B0002' nan '032X1001' '032X1002' '032B1002' '00311001'
 '031A1001']


In [13]:
stops_31A1 = set(df_31[df_31["Journey_Pattern_ID"]=="00311001"].Stop_ID.unique())
stops_31A2 = set(df_31[df_31["Journey_Pattern_ID"]=="00310002"].Stop_ID.unique())
print(sorted(stops_31A1))
print("")
print(sorted(stops_31A2))
print("")
print(stops_31A1 == stops_31A2)

print(stops_31A1 - stops_31A2)
print(stops_31A2 - stops_31A1)
print(stops_31A2.intersection(stops_31A1))
print(len(stops_31A2.union(stops_31A1)) == len(stops_31A2) + len(stops_31A1))
print(df_31[df_31["Journey_Pattern_ID"]=="00310002"].count())

['289']

[]

False
{'289'}
set()
set()
True
Timestamp             0
LineID                0
Direction             0
Journey_Pattern_ID    0
Timeframe             0
Vehicle_Journey_ID    0
Operator              0
Congestion            0
Lon                   0
Lat                   0
Delay                 0
Block_ID              0
Vehicle_ID            0
Stop_ID               0
At_Stop               0
dtype: int64


It seems to me that each journey pattern id is a different route
/slightly modified route.

line ID is somewhat unreliable (but convinient) 
one serve one direction the other serves another

In [2]:
vehicle = df_31.loc(df_31.Vehicle_Journey_ID==2757)&(df.Timeframe="2012-11-05")
vehicle.sort_values("Timestamp")
vehicle



SyntaxError: invalid syntax (<ipython-input-2-ff939f79e26c>, line 1)

cleaning function : Needs to iterate through each file, accumulate rows for speccific routes and clean them and save them as a new csv.

cleaning includes fixing Line_ID
fixing Direction
Dropping nulls
feeds in weather data and attaches the classification columns for dummy variables.

They are then saved again.

we use 





In [18]:
df_31.shape

(114489, 15)

In [19]:
df_31.isnull().sum()

Timestamp             0
LineID                0
Direction             0
Journey_Pattern_ID    2
Timeframe             0
Vehicle_Journey_ID    0
Operator              0
Congestion            0
Lon                   0
Lat                   0
Delay                 0
Block_ID              0
Vehicle_ID            0
Stop_ID               0
At_Stop               0
dtype: int64

In [23]:
df_31[df_31["Journey_Pattern_ID"]=="null"].count()

Timestamp             30876
LineID                30876
Direction             30876
Journey_Pattern_ID    30876
Timeframe             30876
Vehicle_Journey_ID    30876
Operator              30876
Congestion            30876
Lon                   30876
Lat                   30876
Delay                 30876
Block_ID              30876
Vehicle_ID            30876
Stop_ID               30876
At_Stop               30876
dtype: int64

In [ ]:
#alot of these nulls are -re-derivable if we use the signature method